# Homeowrk 1

In [1]:
import astropy.units as u
import astropy.constants as c
from astropy.coordinates import SkyCoord
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import itertools
%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (10, 10)
plt.rc('axes', labelsize=14)
plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=16)
plt.rc('axes', titleweight='bold')
plt.rc('font', family='sans-serif')

## Problem 1: 

The Madelung constant


In condensed matter physics the Madelung constant gives the total electric potential felt by
an atom in a solid. It depends on the charges on the other atoms nearby and their locations.
Consider for instance solid sodium chloride—table salt. The sodium chloride crystal has atoms
arranged on a cubic lattice, but with alternating sodium and chlorine atoms, 

the sodium ones
having a single positive charge +e and the chlorine ones a single negative charge −e, where e is
the charge on the electron. 

If we label each position on the lattice by three integer coordinates
(i, j, k), then **the sodium atoms fall at positions where i + j + k is even, and the chlorine atoms
at positions where i + j + k is odd.**


Consider a sodium atom at the origin, i = j = k = 0, and let us calculate the Madelung
constant. 

If the spacing of atoms on the lattice is a, then the distance from the origin to the atom
at position (i, j, k) is 

$\sqrt{(ia)^2 + (ja)^2 + (ka)^2}= a \sqrt{i^2 + j^2 + k^2}$


and the potential at the origin created by such an atom is

$V(i, j, k) = ± \frac{e}{4 \pi \epsilon_0 a \sqrt{i^2 + j^2 + k^2}} $

with $\epsilon_0$ being the permittivity of the vacuum and the sign of the expression depending on
whether i + j + k is even or odd. 

The **total potential** felt by the sodium atom is then the sum of
this quantity over all other atoms. 

Let us assume a cubic box around the sodium at the origin,
with L atoms in all directions. Then

$V_{total} =  \sum_{i,j,k = -L (not i=j=k=0)}^{L}  V(i,j,k) = \frac{e}{4 \pi \epsilon_0 a}M $


where M is the Madelung constant, at least approximately—technically the Madelung constant
is the value of M when L → ∞, but one can get a good approximation just by using a large
value of L.


Write a program to calculate and print the Madelung constant for sodium chloride. Use as
large a value of L as you can, while still having your program run in reasonable time—say in a
minute or less.

$M = \Sigma  \frac{e}{4\pi \epsilon_0 a \sqrt{i^2 + j^2 + k^2}} x \frac{4 \pi \epsilon_0 a }{e}$

$M = \Sigma \frac{1}{\sqrt{i^2 + j^2 + k^2}}$

In [45]:
def madelung_constant(L):
    """
    Calculates the Madelung constant for a 3-d sqre space of dimmensions 2Lx2Lx2L.
    
    Parameters
    ---
    L: Length of one of the square axes defining the space where the Madelung constant will be calculated
    
    Returns
    ---
    M: The Madelung constant
    """
    #Define I, J, and K axes to run from -L to L
    I=J=K=np.arange(-1*L, L+1)
    #Initialize M at 0
    M = 0
    #Define an empty list to store all possible coordinates within the 3-D space
    coordinates= []
    #Iterate through all possible coordinates in the space
    for i in itertools.product(I,J,K):
        #Append each of these possible coordinates to the list
        coordinates.append(i)
    #Remove the center coordinate
    coordinates.remove((0,0,0))
    #Loop over the possible coordinates
    for coord in coordinates:
        #Check if the sum of the position vectors is even
        if (float(coord[0]+coord[1]+coord[2]))%2 ==0:
            #If it is even, add a negative value
            M+=(1/(np.sqrt((coord[0]**2) + (coord[1]**2) + (coord[2]**2))))*(1)
        #Covering cases where the sum is not even
        else:
            #Add a positive value
            M+=(1/(np.sqrt((coord[0]**2) + (coord[1]**2) + (coord[2]**2))))*(-1)
    #Return the constant
    return M

In [46]:
print(f"The Madelung Constant is {round(madelung_constant(100), 4)}.")

The Madelung Constant is -1.7418.


## Problem 2

 The semi-empirical mass formula
 
 
In nuclear physics, the semi-empirical mass formula is a formula for calculating the approx-
imate nuclear **binding energy B** of an atomic nucleus with **atomic number Z** and **mass num-
ber A**:

$B = a_1 A - a_2 A^{2/3} -a_3 \frac{Z^2}{A^{1/3}} - a_4\frac{(A -2Z)^2}{A}+\frac{a_5}{A^{1/2}}$

where, in units of millions of electron volts, the constants are 
a1 = 15.8, a2 = 18.3, a3 = 0.714,
a4 = 23.2, and

a5=
0 if A is odd,
12.0 if A and Z are both even,
−12.0 if A is even and Z is odd.


a) Write a program that takes as its input the values of A and Z, and prints out the binding
energy for the corresponding atom. Use your program to find the binding energy of an
atom with A = 58 and Z = 28. (Hint: The correct answer is around 490 MeV.)


b) Modify your program to print out not the total binding energy B, but the binding energy
per nucleon, which is B/A.


c) Now modify your program so that it takes as input just a single value of the atomic
number Z and then goes through all values of A from A = Z to A = 3Z, to find the one
that has the largest binding energy per nucleon. This is the most stable nucleus with the
given atomic number. Have your program print out the value of A for this most stable
nucleus and the value of the binding energy per nucleon.
d) Modify your program again so that, instead of taking Z as input, it runs through all val-
ues of Z from 1 to 100 and prints out the most stable value of A for each one. At what
value of Z does the maximum binding energy per nucleon occur? (The true answer, in
real life, is Z = 28, which is nickel

a) Write a program that takes as its input the values of A and Z, and prints out the binding
energy for the corresponding atom. Use your program to find the binding energy of an
atom with A = 58 and Z = 28. (Hint: The correct answer is around 490 MeV.)

In [11]:
def binding_energy(A,Z):
    """
    Computes the binding energy of a specified atom
    
    Parameters
    ---
    A: mass number of atom
    Z: atomic number of atom
    
    Returns
    ---
    B: binding energy in eV of the specified atom
    
    """
    #Assigns values to a1-a4, gives them units of millions of electron volts
    a1 = 15.8e6*u.eV
    a2 = 18.3e6*u.eV
    a3 = 0.714e6*u.eV
    a4 = 23.2e6*u.eV
    #Assigns a5 if A is odd, assigns units of millions of electron volts
    if A%2 !=0:
        a5 = 0*u.eV
    #Assigns a5 if A and Z are even, assigns units of millions of electron volts
    elif A%2 == 0 and Z%2 == 0:
        a5 = 12.0e6*u.eV
    #Assigns a5 if A is even and Z is odd, assigns units of millions of electron volts
    elif A%2 == 0 and Z%2 != 0:
        a5 = -12.0e6*u.eV
    #Computes the binding energy B
    B = (a1*A) - (a2*(A**(2/3))) - (a3*((Z**2)/(A**(1/3)))) - (a4*(((A-2*Z)**2)/(A)))+((a5)/(A**(1/2)))
    #Return the binding energy
    return B

In [12]:
#Running the function to get the binding energy and printing it
binding = binding_energy(A=58,Z=28)
#Converting the binding energy to MeV
print(f'The binding energy is {round(binding.value/1e6, 2)} MeV.')

The binding energy is 497.56 MeV.


This matches the given answer of approximately 490 MeV.

b) Modify your program to print out not the total binding energy B, but the binding energy
per nucleon, which is B/A.

In [13]:
def binding_energy_per_nuc(A,Z):
    """
    Returns the binding energy per nucleon of a specified atom.
    
    Parameters
    ---
    A: mass number of atom
    Z: atomic number of atom
    
    Returns:
    ---
    BE: The binding energy per nucleon for the specified atom
    """
    #Calls a previously defined function to compute the binding energy and divides it by the number of nucleons
    BE = binding_energy(A,Z)/A
    #Returns binding energy per nucleon
    return BE

In [14]:
#Calling the binding_energy_per_nuc function
be = binding_energy_per_nuc(A = 58,Z=28)
#Converting the binding energy per nucleon to MeV
print(f'The binding energy per nucleon is {round(be.value/1e6, 2)} MeV.')

The binding energy per nucleon is 8.58 MeV.


c) Now modify your program so that it takes as input just a single value of the atomic
number Z and then goes through all values of A from A = Z to A = 3Z, to find the one
that has the largest binding energy per nucleon. This is the most stable nucleus with the
given atomic number. Have your program print out the value of A for this most stable
nucleus and the value of the binding energy per nucleon.


In [15]:
def maximize_binding_energy_for_A(Z):
    """
    Computes a range of mass numbers possible for a specified Z between Z and 3Z then determines the 
    mass number that maximizes the binding energy.
    
    Parameters
    ---
    Z: atomic number of atom
    
    Returns
    ---
    maxA: The value for the mass number A associated with the maximum binding energy
    maxBE: The binding energy associated with the A value of maxA.
    """
    #Creates a range of integer values between Z and 3Z
    A_range = np.arange(Z, 3*Z+1)
    #Defines an empty list to store the binding energies
    binding_energies = []
    #Loops over the possible A values
    for A in A_range:
        #Computes the binding energy per nucleon and appends it to the list
        binding_energies.append(binding_energy_per_nuc(A,Z).value)
    #Finds the index value in the binding energies list associated with the highest value and indexes the 
    #A_range array to identifythe A value with the highest binding energy
    maxA = A_range[np.argmax(binding_energies)]
    #Finds the maximum value for the binding energies
    maxBE = np.max(binding_energies)
    #Returns the two maxima
    return maxA, maxBE

In [22]:
maxA, maxBE = maximize_binding_energy_for_A(28)
print(f'The mass number with the maximum binding energy is {maxA}. The binding energyfor this mass number is {round(maxBE/1e6, 3)} MeV.')

The mass number with the maximum binding energy is 62. The binding energyfor this mass number is 8.702 MeV.


d) Modify your program again so that, instead of taking Z as input, it runs through all val- ues of Z from 1 to 100 and prints out the most stable value of A for each one. At what value of Z does the maximum binding energy per nucleon occur? (The true answer, in real life, is Z = 28, which is nickel

In [32]:
def maximize_binding_energy_for_Z():
    """
    Runs through all Z values between 1 and 100 and computes then prints the A value that maximizes the 
    binding energy. Computes the Z value for the atom with the highest binding energy.
    
    Parameters
    ---
    none
    
    Returns
    ---
    maxZ: The Z value associated with the maximum binding energy
    
    """
    #Create a range of Z values between 1 and 100
    Z_range = np.arange(1, 101)
    #Define empty lists to store the maximum A values and their associated binding energies
    maxA = []
    b_es = []
    #Loop through the possible Z values
    for Z in Z_range:
        #Compute the maximum binding energy and associated A value for the given Z
        mxA, mxBE = maximize_binding_energy_for_A(Z)
        #Store the values in their associated lists
        maxA.append(mxA)
        b_es.append(mxBE)
        #Print out the maximum A value for the associated Z
        print(f'For Z = {Z}, the most stable A is {mxA}')
    #Compute the Z value associated with the highest binding energy
    maxZ = Z_range[np.argmax(b_es)]
    #Return the Z value
    return maxZ

In [33]:
mx = maximize_binding_energy_for_Z()
print(f'The most stable binding energy for the first 100 elements occurs at Z = {mx}.')

For Z = 1, the most stable A is 3
For Z = 2, the most stable A is 4
For Z = 3, the most stable A is 7
For Z = 4, the most stable A is 8
For Z = 5, the most stable A is 11
For Z = 6, the most stable A is 14
For Z = 7, the most stable A is 15
For Z = 8, the most stable A is 18
For Z = 9, the most stable A is 19
For Z = 10, the most stable A is 22
For Z = 11, the most stable A is 25
For Z = 12, the most stable A is 26
For Z = 13, the most stable A is 29
For Z = 14, the most stable A is 30
For Z = 15, the most stable A is 33
For Z = 16, the most stable A is 36
For Z = 17, the most stable A is 37
For Z = 18, the most stable A is 40
For Z = 19, the most stable A is 43
For Z = 20, the most stable A is 44
For Z = 21, the most stable A is 47
For Z = 22, the most stable A is 48
For Z = 23, the most stable A is 51
For Z = 24, the most stable A is 54
For Z = 25, the most stable A is 55
For Z = 26, the most stable A is 58
For Z = 27, the most stable A is 61
For Z = 28, the most stable A is 62
For Z

## Problem 3

Binomial coefficients


The binomial coefficient (n
k) is an integer equal to

if k greater than or equal to 1:

$(nk) = \frac{n!}{k!(n-k)!} = \frac{n x (n-1) x (n-2) x ...x(n-k+1)}{1 x 2 x...k}$

else =1 if k = 0

a) Using this form for the binomial coefficient, write a user-defined function binomial(n,k)
that calculates the binomial coefficient for given n and k. Make sure your function returns
the answer in the form of an integer (not a float) and gives the correct value of 1 for the
case where k = 0.
b) Using your function write a program to print out the first 20 lines of “Pascal’s triangle.”
The nth line of Pascal’s triangle contains n + 1 numbers, which are the coefficients (n
0),
(n
1), and so on up to (n
n). Thus the first few lines are
1 1
1 2 1
1 3 3 1
1 4 6 4 1
c) The probability that an unbiased coin, tossed n times, will come up heads k times is
(n
k)/2n. Write a program to calculate (a) the total probability that a coin tossed 100 times
2
comes up heads exactly 60 times, and (b) the probability that it comes up heads 60 or
more tim

a) Using this form for the binomial coefficient, write a user-defined function binomial(n,k)
that calculates the binomial coefficient for given n and k. Make sure your function returns
the answer in the form of an integer (not a float) and gives the correct value of 1 for the
case where k = 0.

In [41]:
def binomial(n, k):
    """
    Calculates the binomial coefficient for a specified n and k
    
    Parameters
    ---
    n: first input for the binomial function, integer
    k: second input for the binomial function, positive integer or 0
    
    Returns
    ---
    result: an integer result for the binomial coefficient
    
    """
    #Check if k is 0
    if k == 0:
        #Return a value of 1 for the binomial coefficient
        return 1
    #Check if the k value is greater than or equal to 1
    elif k>=1:
        #Compute the binomial coefficient and store it in the result
        result =  np.math.factorial(n)/((np.math.factorial(k))*(np.math.factorial(n-k)))
    #Check if the function has been given a non-integer, display an error message if so
    elif (type(k) != int) or (type(n)!= int):
        print('Please specify an integer value.')
    #Catch all other cases where the k value is negative, print an error.
    else:
        print('k value cannot be negative')
    #Return the binomial coefficient as an integer value
    return int(result)

In [43]:
binomial(n = 2, k = 2)

1

In [44]:
binomial(n = 4, k = 3)

4

b) Using your function write a program to print out the first 20 lines of “Pascal’s triangle.”
The nth line of Pascal’s triangle contains n + 1 numbers, which are the coefficients (n
0),
(n
1), and so on up to (n
n). Thus the first few lines are
1 1
1 2 1
1 3 3 1
1 4 6 4 1

In [262]:
def pascals_tri(lines):
    """
    Creates Pascal's triangle for a given number of lines.
    
    Parameters
    ---
    lines: The number of lines for the triangle to be computed
    
    Returns
    ---
    Prints each line of the triangle
    
    """
    #Initialize n at 1
    n=1
    #Loop through each line in a range specefied by the lines variable
    for line in np.arange(1,lines+1):
        #Initializes an empty list to store the numbers in each tear of the iteration
        nums = []
        #Creates values for k for this specified n
        ks = np.arange(0, n+1)
        #Loop through the k values
        for k in ks:
            #Computes the binomial coefficient for each of the k values, stores the result
            nums.append(binomial(n, k))
        #prints the line of Pascal's triangle
        print(nums)
        #Increases n before the next iteration
        n+=1

In [263]:
pascals_tri(20)

[1, 1]
[1, 2, 1]
[1, 3, 3, 1]
[1, 4, 6, 4, 1]
[1, 5, 10, 10, 5, 1]
[1, 6, 15, 20, 15, 6, 1]
[1, 7, 21, 35, 35, 21, 7, 1]
[1, 8, 28, 56, 70, 56, 28, 8, 1]
[1, 9, 36, 84, 126, 126, 84, 36, 9, 1]
[1, 10, 45, 120, 210, 252, 210, 120, 45, 10, 1]
[1, 11, 55, 165, 330, 462, 462, 330, 165, 55, 11, 1]
[1, 12, 66, 220, 495, 792, 924, 792, 495, 220, 66, 12, 1]
[1, 13, 78, 286, 715, 1287, 1716, 1716, 1287, 715, 286, 78, 13, 1]
[1, 14, 91, 364, 1001, 2002, 3003, 3432, 3003, 2002, 1001, 364, 91, 14, 1]
[1, 15, 105, 455, 1365, 3003, 5005, 6435, 6435, 5005, 3003, 1365, 455, 105, 15, 1]
[1, 16, 120, 560, 1820, 4368, 8008, 11440, 12870, 11440, 8008, 4368, 1820, 560, 120, 16, 1]
[1, 17, 136, 680, 2380, 6188, 12376, 19448, 24310, 24310, 19448, 12376, 6188, 2380, 680, 136, 17, 1]
[1, 18, 153, 816, 3060, 8568, 18564, 31824, 43758, 48620, 43758, 31824, 18564, 8568, 3060, 816, 153, 18, 1]
[1, 19, 171, 969, 3876, 11628, 27132, 50388, 75582, 92378, 92378, 75582, 50388, 27132, 11628, 3876, 969, 171, 19, 1]
[1, 2

c) The probability that an unbiased coin, tossed n times, will come up heads k times is
(n
k)/2n. Write a program to calculate (a) the total probability that a coin tossed 100 times
2
comes up heads exactly 60 times,

In [47]:
def probability_heads(tosses, heads):
    """
    Computes the probability of rolling heads a specified ammount of times for a specified number of tosses
    
    Parameters
    ---
    tosses: the number of tosses 
    
    heads: the number of heads occuring in the number of tosses
    
    Returns
    ---
    prints the probabilities
    
    """
    print(f"The probability of rolling {heads} heads for {tosses} tosses is {round(((binomial(tosses, heads)/(2**(tosses))))*100, 3)}%.")

In [51]:
prob = probability_heads(tosses=100, heads=60)

The probability of rolling 60 heads for 100 tosses is 1.084%.


(b) the probability that it comes up heads 60 or
more times

In [280]:
def probaility_more_heads(tosses, min_heads):
    """
    Computes the probability for at least the specified number of tosses occuring for a given number of tosses
    
    Parameters
    ---
    tosses: the number of tosses 
    
    min_heads: the minimum number of heads rolled
    
    Returns
    ---
    prints the percent probability
    
    """
    #Initialize a variable to track the probability
    h_prob = 0
    #Loop through the minimum number of tosses to the total tosses
    for h in np.arange(min_heads, tosses +1):
        #Increase the probability sum for each iteration
        h_prob += (binomial(tosses, h)/(2**(tosses)))
    #Prints the percent probability
    print(f'The probability for flipping {min_heads} or more heads for {tosses} coin tosses is {round(h_prob*100, 3)}%.')

In [281]:
probaility_more_heads(100, 60)

The probability for flipping 60 or more heads for 100 coin tosses is 2.844%.


## Problem 4

Catalan numbers

The Catalan numbers Cn are a sequence of integers 1, 1, 2, 5, 14, 42, 132. . . that play an important
role in quantum mechanics and the theory of disordered systems. (They were central to Eugene
Wigner’s proof of the so-called semicircle law.) They are given by

$C_0 = 1$

$C_{n+1} = \frac{4n +2}{n+2}C_n$


Write a program that prints in increasing order all Catalan numbers less than or equal to one
billion

In [294]:
def catalan_numbers():
    """
    Prints the Catalan numbes in increasing order up to one billion
    
    Parameters
    ---
    none
    
    Returns
    ---
    prints Catalan numbers
    
    """
    #Printing c0
    print(1)
    #Initializing c as the current catalan number and n as an iteration tracker
    c = 1
    n = 0
    #Printing c1
    print(c)
    #Continues the loop while the catalan number is less than one billion
    while c<=1e9:
        #Increases the iteration tracker
        n+=1
        #Increases the catalan number by multiplying the previous catalan number by the iterative fraction
        c*=((4*n+2)/(n+2))
        #Prints the new catalan number
        print(c)

In [295]:
catalan_numbers()

1
1
2.0
5.0
14.0
42.0
132.0
429.0
1430.0
4862.0
16796.0
58786.0
208012.0
742900.0
2674440.0
9694845.0
35357670.0
129644790.0
477638700.0
1767263190.0


## Problem 5

Recursion
A useful feature of user-defined functions is recursion, the ability of a function to call itself. For
example, consider the following definition of the factorial n! of a positive integer n:
n! =
{ 1 if n = 1,
n × (n − 1)! if n > 1.
This constitutes a complete definition of the factorial which allows us to calculate the value of
n! for any positive integer. We can employ this definition directly to create a Python function
for factorials, like this:
def factorial(n):
if n==1:
return 1
else:
return n*factorial(n-1)
Note how, if n is not equal to 1, the function calls itself to calculate the factorial of n − 1.
This is recursion. If we now say “print(factorial(5))” the computer will correctly print the
answer 120.
a) We encountered the Catalan numbers Cn in Problem 4. With just a little rearrangement,
the definition given there can be rewritten in the form
Cn =



1 if n = 0,
4n − 2
n + 1 Cn−1 if n > 0.
Write a Python function, using recursion, that calculates Cn. Use your function to calcu-
late and print C100.
3
b) Euclid showed that the greatest common divisor g(m, n) of two nonnegative integers m
and n satisfies
g(m, n) =
{ m if n = 0,
g(n, m mod n) if n > 0.
Write a Python function g(m,n) that employs recursion to calculate the greatest common
divisor of m and n using this formula. Use your function to calculate and print the great-
est common divisor of 108 and 192.
Comparing the calculation of the Catalan numbers in Problem 4 and here, we see that it’s possi-
ble to do the calculation two ways, either directly or using recursion. In most cases, if a quantity
can be calculated without recursion, then it will be faster to do so, and we normally recommend
taking this route if possible. There are some calculations, however, that are essentially impos-
sible (or at least much more difficult) without recursion. We will see some examples later in
the bo

Write a Python function, using recursion, that calculates Cn. Use your function to calcu-
late and print C100.
3

In [53]:
def catlan_recursion(n):
    """
    Returns the nth Catalan number
    
    Parameters
    ---
    n: integer, indicates catalan number of interest
    
    Returns
    ---
    returns the specified catalan number
    
    """
    #Check if n  is 0, if so return 1
    if n ==0:
        return 1
    #If n is not 1, call the function inside itsself recursively and multiply by the fraction for catlan numbers
    else:
        return ((4*n-2)/(n+1))*catlan_recursion(n-1)

In [56]:
print(f"The 100th Catalan number is {catlan_recursion(100):.3e}.")

The 100th Catalan number is 8.965e+56.


b) Euclid showed that the greatest common divisor g(m, n) of two nonnegative integers m
and n satisfies
g(m, n) =
{ m if n = 0,
g(n, m mod n) if n > 0.
Write a Python function g(m,n) that employs recursion to calculate the greatest common
divisor of m and n using this formula. Use your function to calculate and print the great-
est common divisor of 108 and 192.

In [57]:
def common_divisor(m,n):
    """
    Returns the greatest common divisor of m and n
    
    Parameters:
    ---
    m: integer
    
    n: integer
    
    Returns:
    ---
    returns the greatest common divisor
    
    """
    #Check if n is 0, if so return m
    if n == 0:
        return m
    #Check that both numbers are integers, print an error if they are not
    elif type(m) != int or type(n)!= int:
        print('Please enter an integer.')
    #Check that n is greater than zero, run the function recursively with n input for m and m%n input for n
    elif n>0:
        return common_divisor(n, m%n)
    #For all other cases n is negative, print an error 
    else:
        print('n must be greater or euqal to 0.')

In [61]:
print(f'The greatest common divisor for 108 and 192 is {common_divisor(108, 192)}.')

The greatest common divisor for 108 and 192 is 12.
